In [284]:
import random
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Set the start and end date for the dataset (5 years from today)
start_date = datetime.now()
end_date = start_date + timedelta(days=10 * 365)

# Define the hospital, department, and admission count patterns
hospitals = ['St. Mary Hospital', 'General Medical Center', 'City Health Center', 'Hope Medical Center']
departments = ['Pediatrics', 'General', 'Cardiology', 'Oncology', 'Neurology']

HW = [5,10,15,20,25]

HospitalWeight = {i:j for i,j in zip(hospitals,HW)}


# Generate the dataset
dataset = []
current_date = start_date


while current_date <= end_date:
    for hospital in hospitals:
        for department in departments:
            day_of_week = current_date.weekday()  # 0 = Monday, 6 = Sunday
            day_of_month = current_date.day
            
            if department == "General":
                
                # Apply pattern based on day of the week and day of the month
                admission_count = (60 + 10 * day_of_week - day_of_month) + HospitalWeight[hospital]
                admission_count = max(0, admission_count)  # Ensure non-negative values
                
                
            if department == "Pediatrics":
                
                # Apply pattern based on day of the week and day of the month
                admission_count = (40 + 10 * day_of_week - (day_of_month)/2) + HospitalWeight[hospital]
                admission_count = max(0, admission_count)  # Ensure non-negative values
                
            if department == "Cardiology":
                
                # Apply pattern based on day of the week and day of the month
                admission_count = (25 + 10 * day_of_week - (day_of_month)/3)+ HospitalWeight[hospital]
                admission_count = max(0, admission_count)  # Ensure non-negative values
                
            if department == "Oncology":
                
                # Apply pattern based on day of the week and day of the month
                admission_count = (15 + 10 * day_of_week - (day_of_month)/4) + HospitalWeight[hospital]
                admission_count = max(0, admission_count)  # Ensure non-negative values
                
            if department == "Oncology":
                
                # Apply pattern based on day of the week and day of the month
                admission_count = (5 + 10 * day_of_week - (day_of_month/5)) + HospitalWeight[hospital]
                admission_count = max(0, admission_count)  # Ensure non-negative values
                
            discharge_count = int(admission_count/2)
            
            transfer_count = int(admission_count/6)
            
            dataset.append({
                'Date': current_date,
                'Hospital': hospital,
                'Department': department,
                'AdmissionCount': admission_count,
                'DischargeCount': discharge_count,
                'TransferCount': transfer_count
            })

    current_date += timedelta(days=1)

# Create pandas DataFrame from the dataset
df = pd.DataFrame(dataset)


# Save the dataset as CSV
filename = '..\data\patient_admissions.csv'
df.to_csv(filename, index=False)
print(f"Dataset stored as {filename}")


Dataset stored as ..\data\patient_admissions.csv


In [1]:
import pandas as pd

data = pd.read_csv("..\data\patient_admissions.csv")

In [2]:
data.columns

Index(['Date', 'Hospital', 'Department', 'AdmissionCount', 'DischargeCount',
       'TransferCount'],
      dtype='object')

In [286]:
import pandas as pd

def create_date_features(dataframe, date_column):
    # Convert the date column to datetime type
    dataframe[date_column] = pd.to_datetime(dataframe[date_column],dayfirst=True)
    
    dataframe['day_of_week'] = dataframe[date_column].dt.weekday
    dataframe['Day'] = dataframe[date_column].dt.day
    
    return dataframe


In [291]:
data = create_date_features(data,"Date")

In [292]:
data

,Date,Hospital,Department,AdmissionCount,DischargeCount,TransferCount,day_of_week,Day
0,2023-07-10 11:22:37.241052,St. Mary Hospital,Pediatrics,40.000000,20,6,0,10
1,2023-07-10 11:22:37.241052,St. Mary Hospital,General,55.000000,27,9,0,10
2,2023-07-10 11:22:37.241052,St. Mary Hospital,Cardiology,26.666667,13,4,0,10
3,2023-07-10 11:22:37.241052,St. Mary Hospital,Oncology,8.000000,4,1,0,10
4,2023-07-10 11:22:37.241052,St. Mary Hospital,Neurology,8.000000,4,1,0,10
...,...,...,...,...,...,...,...,...
73015,2033-07-07 11:22:37.241052,Hope Medical Center,Pediatrics,86.500000,43,14,3,7
73016,2033-07-07 11:22:37.241052,Hope Medical Center,General,103.000000,51,17,3,7
73017,2033-07-07 11:22:37.241052,Hope Medical Center,Cardiology,72.666667,36,12,3,7
73018,2033-07-07 11:22:37.241052,Hope Medical Center,Oncology,53.600000,26,8,3,7


In [293]:
cat_coumns = ["Hospital","Department"]

num_column =["day_of_week","Day","AdmissionCount","DischargeCount","TransferCount"]

dummies = pd.get_dummies(data[cat_coumns],drop_first=True)

data_new = pd.concat([dummies,data[num_column]],axis=1)

In [294]:
data_new.columns

Index(['Hospital_General Medical Center', 'Hospital_Hope Medical Center',
       'Hospital_St. Mary Hospital', 'Department_General',
       'Department_Neurology', 'Department_Oncology', 'Department_Pediatrics',
       'day_of_week', 'Day', 'AdmissionCount', 'DischargeCount',
       'TransferCount'],
      dtype='object')

In [295]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Splitting data into train test split

target,features = data_new[["AdmissionCount","DischargeCount","TransferCount"]],data_new.drop(["AdmissionCount","DischargeCount","TransferCount"],axis=1)

features = features.astype("int")

x_train, x_test, y_train, y_test = train_test_split(features, target)

model = XGBRegressor()

model.fit(x_train,y_train)

### Evaluating the model

from sklearn.metrics import mean_absolute_error, r2_score,mean_squared_error

predicted = model.predict(x_test)

print("R2 Score is : ",r2_score(predicted,y_test))

print("Mean Squared Error is : ",mean_squared_error(predicted,y_test))

print("Mean Absolute Error is : ",mean_absolute_error(predicted,y_test))

model.save_model(r"..\models\admission_model.model")

R2 Score is :  0.99942818880822
Mean Squared Error is :  0.03702559104263121
Mean Absolute Error is :  0.13717511907771904


In [296]:
model.predict(x_train.iloc[15:16])

array([[87.25014 , 43.47673 , 14.062789]], dtype=float32)

In [297]:
x_train.iloc[15:16:]

,Hospital_General Medical Center,Hospital_Hope Medical Center,Hospital_St. Mary Hospital,Department_General,Department_Neurology,Department_Oncology,Department_Pediatrics,day_of_week,Day
20376,0,1,0,1,0,0,0,3,23


In [298]:
y_train.iloc[15:16,:]

,AdmissionCount,DischargeCount,TransferCount
20376,87.0,43,14


In [299]:
data["AdmissionCount"].describe()


count    73020.000000
mean        62.455081
std         27.056767
min          3.800000
25%         42.666667
50%         62.333333
75%         81.333333
max        139.000000
Name: AdmissionCount, dtype: float64

In [300]:
data["DischargeCount"].describe()

count    73020.000000
mean        30.836127
std         13.574277
min          1.000000
25%         21.000000
50%         31.000000
75%         40.000000
max         69.000000
Name: DischargeCount, dtype: float64

In [301]:
data["TransferCount"].describe()

count    73020.000000
mean         9.945714
std          4.533006
min          0.000000
25%          7.000000
50%         10.000000
75%         13.000000
max         23.000000
Name: TransferCount, dtype: float64

In [302]:
data.shape

(73020, 8)

In [303]:
# Making prediction

In [347]:
def get_admission_sample():

    sample_admission ={'Hospital_General Medical Center': [0],
     'Hospital_Hope Medical Center': [0],
     'Hospital_St. Mary Hospital': [0],
     'Department_General': [0],
     'Department_Neurology': [0],
     'Department_Oncology': [0],
     'Department_Pediatrics': [0],
     'day_of_week': [0],
     'Day': [0]}
    
    return sample_admission

In [348]:
sample_admission.keys()

dict_keys(['Hospital_General Medical Center', 'Hospital_Hope Medical Center', 'Hospital_St. Mary Hospital', 'Department_General', 'Department_Neurology', 'Department_Oncology', 'Department_Pediatrics', 'day_of_week', 'Day'])

In [392]:
def prepare_admission_sample(current_date,department):
    
    sample_admission = get_admission_sample()
    
    sample_admission['day_of_week'] = current_date.weekday() 
    sample_admission['Day'] = current_date.day
    if department =="Department_Cardiology":
        
        pass
        
    else:
            
        sample_admission[department] = [1]

        
    return pd.DataFrame(sample_admission)


In [393]:
admission_input = prepare_admission_sample(next_few_days[0],departments[0])

In [394]:
model.predict(admission_input)[0]

array([73.77909 , 36.880325, 11.728316], dtype=float32)

In [424]:
def get_next_fewdays_ADT():

    admission_output ={

        "Next day":{
        "Department_General":0,
        "Department_Neurology":0,
        "Department_Oncology":0,
        "Department_Pediatrics":0,
                    },
        "Next 2 days":{
        "Department_General":0,
        "Department_Neurology":0,
        "Department_Oncology":0,
        "Department_Pediatrics":0,
                    },
        "Next 3 days":{
        "Department_General":0,
        "Department_Neurology":0,
        "Department_Oncology":0,
        "Department_Pediatrics":0,
                    }
    }

    discharge_output ={

        "Next day":{
        "Department_General":0,
        "Department_Neurology":0,
        "Department_Oncology":0,
        "Department_Pediatrics":0,
                    },
        "Next 2 days":{
        "Department_General":0,
        "Department_Neurology":0,
        "Department_Oncology":0,
        "Department_Pediatrics":0,
                    },
        "Next 3 days":{
        "Department_General":0,
        "Department_Neurology":0,
        "Department_Oncology":0,
        "Department_Pediatrics":0,
                    }
    }

    transfer_output ={

        "Next day":{
        "Department_General":0,
        "Department_Neurology":0,
        "Department_Oncology":0,
        "Department_Pediatrics":0,
                    },
        "Next 2 days":{
        "Department_General":0,
        "Department_Neurology":0,
        "Department_Oncology":0,
        "Department_Pediatrics":0,
                    },
        "Next 3 days":{
        "Department_General":0,
        "Department_Neurology":0,
        "Department_Oncology":0,
        "Department_Pediatrics":0,
                    }
    }


    from datetime import datetime

    from datetime import timedelta

    today = datetime.today()

    tomorrow = today+timedelta(days=1)

    day_after_tomorrow = today+timedelta(days=2)

    next_to_DFT = today+timedelta(days=3)

    departments = ['Department_General', 'Department_Neurology', 'Department_Oncology', 'Department_Pediatrics','Department_Cardiology']

    next_few_days = [tomorrow,day_after_tomorrow,next_to_DFT]


    for ind,date in enumerate(next_few_days):

        for department in departments:

            admission_input = prepare_admission_sample(date,department)

            Admission,Discharge,Transfer = [int(i) for i in model.predict(admission_input)[0]]

            for output,prediction in zip([admission_output,discharge_output,transfer_output],[Admission,Discharge,Transfer]):

                if department in output[list(output.keys())[ind]].keys():

                    output[list(output.keys())[ind]][department] = prediction

                else:

                    output[list(output.keys())[ind]]['Department_Cardiology'] = prediction
                    
    return admission_output,discharge_output,transfer_output


In [416]:
def get_total_admission(admission_output):
    
    output = {}
    
    for day in admission_output.keys():
        
        total = 0
        
        for department in admission_output[day].keys():
            
            total+=admission_output[day][department]
            
        output[day] = total
        
    return output

In [422]:
get_total_admission(transfer_output)

{'Next day': 33, 'Next 2 days': 41, 'Next 3 days': 50}